<a href="https://colab.research.google.com/github/talhaanwarch/OffenseEval2020/blob/master/submissions/Danish.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install focal-loss
!pip install keras-tcn==2.8.3

In [2]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Import lib

In [3]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential,Model
from keras.layers import Embedding,CuDNNGRU,CuDNNLSTM,Dense,Dropout,Bidirectional,BatchNormalization,GlobalMaxPooling1D,Flatten, GlobalAveragePooling1D, MaxPooling1D,SpatialDropout1D,Input,Activation,concatenate,Conv1D
from keras.optimizers import RMSprop,Adam,Adadelta
from keras.initializers import Constant
from sklearn.utils import class_weight
from sklearn.metrics import classification_report,f1_score
import keras
import collections
import numpy as np
from focal_loss import BinaryFocalLoss
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)


Using TensorFlow backend.


# Load data

In [4]:
cd /content/drive/My Drive/dataset/OffenseEval2020/data/Danish/

/content/drive/My Drive/dataset/OffenseEval2020/data/Danish


In [5]:
ls

cc.da.300.vec     offenseval-da-test-v1-nolabels.tsv
cc.da.300.vec.gz  offenseval-da-training-v1.tsv


In [6]:
train=pd.read_csv( 'offenseval-da-training-v1.tsv',sep="\t")
train.drop(train.tail(n=1).index,inplace=True)
train["tweet"]= train["tweet"].str.replace('@USER', "") 
train['tweet']=train['tweet'].str.replace('\d+', '')
train['tweet']=train['tweet'].str.replace('URL', '')
train["tweet"]= train["tweet"].str.lower()
train=train.sample(frac=1)
print(len(train))
train.head()

2960


,id,tweet,subtask_a
2291,3,fuck første maj. busserne kører ændret rute og...,OFF
1374,3356,[så los mig hårdt i bollerne](,NOT
544,778,hvad sker der oppe i hovedet på folk !!!,NOT
2134,2193,risengrøden skal have lurpark på. og kanelsukk...,NOT
2021,3080,amager ser godt nok spøjs ud.,NOT


In [7]:
test=pd.read_csv( 'offenseval-da-test-v1-nolabels.tsv',sep='\t')
test["tweet"]= test["tweet   subtask_a"].str.replace('@USER', "") 
test['tweet']=test['tweet'].str.replace('\d+', '')
test['tweet']=test['tweet'].str.replace('URL', '')
test['tweet']=test['tweet'].str.lower()
ids=test['id']
test=test['tweet']
test.head()

0    der er syriske "flygtninge" som rejser til ira...
1                               danmark = vitryssland?
2    ja tvangsfjernelser af børn på urigtige oplysn...
3    han kan ikke svensk og forventer et job. hvis ...
4                                    ned med svensken!
Name: tweet, dtype: object

#Label Encoding

In [0]:
labels=train['subtask_a']
train=train['tweet']

In [9]:
le=LabelEncoder()
labels=le.fit_transform(labels)
print(len(labels))

2960


# Common Parameters

In [0]:
test_size=0.15
batch_size=32
opt=Adam()

## Cylic learning rate

In [0]:
# https://www.kaggle.com/hireme/fun-api-keras-f1-metric-cyclical-learning-rate/code
from keras.callbacks import Callback
class CyclicLR(Callback):
    """This callback implements a cyclical learning rate policy (CLR).
    The method cycles the learning rate between two boundaries with
    some constant frequency, as detailed in this paper (https://arxiv.org/abs/1506.01186).
    The amplitude of the cycle can be scaled on a per-iteration or 
    per-cycle basis.
    This class has three built-in policies, as put forth in the paper.
    "triangular":
        A basic triangular cycle w/ no amplitude scaling.
    "triangular2":
        A basic triangular cycle that scales initial amplitude by half each cycle.
    "exp_range":
        A cycle that scales initial amplitude by gamma**(cycle iterations) at each 
        cycle iteration.
    For more detail, please see paper.
    
    # Example
        ```python
            clr = CyclicLR(base_lr=0.001, max_lr=0.006,
                                step_size=2000., mode='triangular')
            model.fit(X_train, Y_train, callbacks=[clr])
        ```
    
    Class also supports custom scaling functions:
        ```python
            clr_fn = lambda x: 0.5*(1+np.sin(x*np.pi/2.))
            clr = CyclicLR(base_lr=0.001, max_lr=0.006,
                                step_size=2000., scale_fn=clr_fn,
                                scale_mode='cycle')
            model.fit(X_train, Y_train, callbacks=[clr])
        ```    
    # Arguments
        base_lr: initial learning rate which is the
            lower boundary in the cycle.
        max_lr: upper boundary in the cycle. Functionally,
            it defines the cycle amplitude (max_lr - base_lr).
            The lr at any cycle is the sum of base_lr
            and some scaling of the amplitude; therefore 
            max_lr may not actually be reached depending on
            scaling function.
        step_size: number of training iterations per
            half cycle. Authors suggest setting step_size
            2-8 x training iterations in epoch.
        mode: one of {triangular, triangular2, exp_range}.
            Default 'triangular'.
            Values correspond to policies detailed above.
            If scale_fn is not None, this argument is ignored.
        gamma: constant in 'exp_range' scaling function:
            gamma**(cycle iterations)
        scale_fn: Custom scaling policy defined by a single
            argument lambda function, where 
            0 <= scale_fn(x) <= 1 for all x >= 0.
            mode paramater is ignored 
        scale_mode: {'cycle', 'iterations'}.
            Defines whether scale_fn is evaluated on 
            cycle number or cycle iterations (training
            iterations since start of cycle). Default is 'cycle'.
    """

    def __init__(self, base_lr=0.001, max_lr=0.006, step_size=2000., mode='triangular',
                 gamma=1., scale_fn=None, scale_mode='cycle'):
        super(CyclicLR, self).__init__()

        self.base_lr = base_lr
        self.max_lr = max_lr
        self.step_size = step_size
        self.mode = mode
        self.gamma = gamma
        if scale_fn == None:
            if self.mode == 'triangular':
                self.scale_fn = lambda x: 1.
                self.scale_mode = 'cycle'
            elif self.mode == 'triangular2':
                self.scale_fn = lambda x: 1/(2.**(x-1))
                self.scale_mode = 'cycle'
            elif self.mode == 'exp_range':
                self.scale_fn = lambda x: gamma**(x)
                self.scale_mode = 'iterations'
        else:
            self.scale_fn = scale_fn
            self.scale_mode = scale_mode
        self.clr_iterations = 0.
        self.trn_iterations = 0.
        self.history = {}

        self._reset()

    def _reset(self, new_base_lr=None, new_max_lr=None,
               new_step_size=None):
        """Resets cycle iterations.
        Optional boundary/step size adjustment.
        """
        if new_base_lr != None:
            self.base_lr = new_base_lr
        if new_max_lr != None:
            self.max_lr = new_max_lr
        if new_step_size != None:
            self.step_size = new_step_size
        self.clr_iterations = 0.
        
    def clr(self):
        cycle = np.floor(1+self.clr_iterations/(2*self.step_size))
        x = np.abs(self.clr_iterations/self.step_size - 2*cycle + 1)
        if self.scale_mode == 'cycle':
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(cycle)
        else:
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(self.clr_iterations)
        
    def on_train_begin(self, logs={}):
        logs = logs or {}

        if self.clr_iterations == 0:
            K.set_value(self.model.optimizer.lr, self.base_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.clr())        
            
    def on_batch_end(self, epoch, logs=None):
        
        logs = logs or {}
        self.trn_iterations += 1
        self.clr_iterations += 1

        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        self.history.setdefault('iterations', []).append(self.trn_iterations)

        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
        
        K.set_value(self.model.optimizer.lr, self.clr())

In [0]:
clr = CyclicLR(base_lr=0.001, max_lr=0.005,
                        step_size=4., mode='exp_range',
                        gamma=0.99994)

# Tokenize data

In [0]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
max_words = 10000 #frequency of words to be kept
max_len = 200

tokenize = Tokenizer(num_words=max_words)
tokenize.fit_on_texts(train)
sequences = tokenize.texts_to_sequences(train)
word_index = tokenize.word_index
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len,padding='post')

In [14]:
len(sequences_matrix),len(labels)

(2960, 2960)

In [15]:
num_words = min(max_words, len(word_index)) + 1
print(num_words)

9802


In [0]:
test_sequences = tokenize.texts_to_sequences(test)
X_test_sequences = sequence.pad_sequences(test_sequences,maxlen=max_len,padding='post')

In [17]:
len(X_test_sequences)

329

# CharLevel Tokenization

In [0]:
#https://github.com/BrambleXu/nlp-beginner-guide-keras/blob/master/char-level-cnn/char_cnn.py
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.preprocessing.sequence import pad_sequences

max_words = 10000 #frequency of words to be kept
max_len = 200
#a	b	c	d	e	f	g	h	i	j	k	l	m	n	o	p	q	r	s	t	u	v	w	x	y	z	æ	ø	å
ch_tokenize = Tokenizer(num_words=None, char_level=True, oov_token='UNK')
ch_tokenize.fit_on_texts(train)

alphabet = "abcdefghijklmnopqrstuvwxyzæøå,;.!?:'\"/\\|_@#$%^&*~`+-=<>()[]{}"
char_dict = {}
for i, char in enumerate(alphabet):
    char_dict[char] = i + 1

# Use char_dict to replace the ch_tokenize.word_index
ch_tokenize.word_index = char_dict.copy()
# Add 'UNK' to the vocabulary
ch_tokenize.word_index[ch_tokenize.oov_token] = max(char_dict.values()) + 1

train_sequences = ch_tokenize.texts_to_sequences(train)
text_sequences = ch_tokenize.texts_to_sequences(test)

# Padding
train_data = pad_sequences(train_sequences, maxlen=max_len, padding='post')
test_data = pad_sequences(text_sequences, maxlen=max_len, padding='post')

# Embeddings

In [0]:
embed_size=300
def get_coefs(word,*arr):
    return word, np.asarray(arr, dtype='float32')

def build_matrix(embedding_path, word_index):
    embedding_index = dict(get_coefs(*o.strip().split(" ")) for o in open(embedding_path))

    nb_words = min(max_words, len(word_index))
    embedding_matrix = np.zeros((nb_words + 1, embed_size))
    for word, i in word_index.items():
        if i >= max_words:
            continue
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

In [20]:
embeddings=build_matrix('cc.da.300.vec', word_index)
embeddings.shape

(9802, 300)

# Base Models

In [0]:
from sklearn.model_selection import train_test_split
x_train,x_val,y_train,y_val=train_test_split(train,labels,test_size=test_size,stratify=labels,shuffle=True,random_state=2020)

In [22]:
collections.Counter(y_val)

Counter({0: 386, 1: 58})

## Count Vectorizer

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer()
count_vectorizer.fit(x_train)
X_train_cv = count_vectorizer.transform(x_train)
X_val_cv  = count_vectorizer.transform(x_val)
X_test_cv=  count_vectorizer.transform(test)

In [24]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
def logistic_param_selection(X, y, nfolds):
    C= [0.1, 1,3,5,8, 10,12,15]
    param_grid = {'C': C}
    grid_search = GridSearchCV(LogisticRegression(max_iter=1000), param_grid, cv=nfolds,scoring='f1_macro')
    grid_search.fit(X, y)
    print(grid_search.best_params_)
    return grid_search.best_score_
count_vectorizer.fit(train)
train_cv = count_vectorizer.transform(train)
logistic_param_selection(train_cv,labels,5)

{'C': 15}


0.7111039764583682

In [25]:
from sklearn.linear_model import LogisticRegression
cv_classifier = LogisticRegression(solver='lbfgs',C=15,max_iter=500,class_weight='balance')
cv_classifier.fit(X_train_cv, y_train)
y_pred = cv_classifier.predict(X_val_cv)
print(classification_report(y_val, y_pred))
cv_f1=f1_score(y_val, y_pred,average='macro')

              precision    recall  f1-score   support

           0       0.92      0.97      0.95       386
           1       0.73      0.47      0.57        58

    accuracy                           0.91       444
   macro avg       0.83      0.72      0.76       444
weighted avg       0.90      0.91      0.90       444



## TF IDF word vectorizer

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    ngram_range=(1, 4),
    max_features=10000)
word_vectorizer.fit(x_train)
train_word_features = word_vectorizer.transform(x_train)
val_word_features = word_vectorizer.transform(x_val)
test_word_features = word_vectorizer.transform(test)

In [27]:
from sklearn.linear_model import LogisticRegression
tfw_classifier = LogisticRegression(solver='lbfgs',max_iter=500)
tfw_classifier.fit(train_word_features, y_train)
y_pred = tfw_classifier.predict(val_word_features)
print(classification_report(y_val, y_pred))
tfw_f1=f1_score(y_val, y_pred,average='macro')

              precision    recall  f1-score   support

           0       0.88      1.00      0.93       386
           1       1.00      0.05      0.10        58

    accuracy                           0.88       444
   macro avg       0.94      0.53      0.52       444
weighted avg       0.89      0.88      0.82       444



## TF IDF char vectorizer

In [0]:
char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    ngram_range=(1, 6),
    max_features=30000)
char_vectorizer.fit(x_train)
train_char_features = char_vectorizer.transform(x_train)
val_char_features = char_vectorizer.transform(x_val)
test_char_features = char_vectorizer.transform(test)

In [29]:
tfc_classifier = LogisticRegression(solver='lbfgs',max_iter=500)
tfc_classifier.fit(train_char_features, y_train)
y_pred = tfc_classifier.predict(val_char_features)
print(classification_report(y_val, y_pred))
tfc_f1=f1_score(y_val, y_pred,average='macro')

              precision    recall  f1-score   support

           0       0.88      1.00      0.93       386
           1       1.00      0.07      0.13        58

    accuracy                           0.88       444
   macro avg       0.94      0.53      0.53       444
weighted avg       0.89      0.88      0.83       444



# Models

In [0]:
from sklearn.model_selection import train_test_split
x_train,x_val,y_train,y_val=train_test_split(sequences_matrix,labels,test_size=test_size,stratify=labels,shuffle=True,random_state=2020)

##GRU

In [0]:
def model_gru():
  inp = Input(shape=(max_len,))
  x = Embedding(num_words,embed_size,embeddings_initializer=Constant(embeddings),input_length=max_len,trainable=False)(inp)
  x = SpatialDropout1D(0.1)(x)
  x = Bidirectional(CuDNNLSTM(50, return_sequences=True))(x)
  x, x_h, x_c = Bidirectional(CuDNNGRU(50, return_sequences=True, return_state = True))(x)
  avg_pool = GlobalAveragePooling1D()(x)
  max_pool = GlobalMaxPooling1D()(x)
  conc = concatenate([avg_pool, x_h, max_pool])
  outp = Dense(1, activation="sigmoid")(conc)    
  model = Model(inputs=inp, outputs=outp)
  model.compile(loss=BinaryFocalLoss(gamma=2), optimizer=opt, metrics=['accuracy'])
  return model

## GRU attention

In [0]:
#https://github.com/zake7749/DeepToxic
from keras.layers import Layer,Lambda
from keras import initializers
from keras.engine import InputSpec, Layer
from keras import backend as K

class AttentionWeightedAverage(Layer):
    """
    Computes a weighted average of the different channels across timesteps.
    Uses 1 parameter pr. channel to compute the attention value for a single timestep.
    """

    def __init__(self, return_attention=False, **kwargs):
        self.init = initializers.get('uniform')
        self.supports_masking = True
        self.return_attention = return_attention
        super(AttentionWeightedAverage, self).__init__(** kwargs)

    def build(self, input_shape):
        self.input_spec = [InputSpec(ndim=3)]
        assert len(input_shape) == 3

        self.W = self.add_weight(shape=(input_shape[2], 1),
                                 name='{}_W'.format(self.name),
                                 initializer=self.init)
        self.trainable_weights = [self.W]
        super(AttentionWeightedAverage, self).build(input_shape)

    def call(self, x, mask=None):
        # computes a probability distribution over the timesteps
        # uses 'max trick' for numerical stability
        # reshape is done to avoid issue with Tensorflow
        # and 1-dimensional weights
        logits = K.dot(x, self.W)
        x_shape = K.shape(x)
        logits = K.reshape(logits, (x_shape[0], x_shape[1]))
        ai = K.exp(logits - K.max(logits, axis=-1, keepdims=True))

        # masked timesteps have zero weight
        if mask is not None:
            mask = K.cast(mask, K.floatx())
            ai = ai * mask
        att_weights = ai / (K.sum(ai, axis=1, keepdims=True) + K.epsilon())
        weighted_input = x * K.expand_dims(att_weights)
        result = K.sum(weighted_input, axis=1)
        if self.return_attention:
            return [result, att_weights]
        return result

    def get_output_shape_for(self, input_shape):
        return self.compute_output_shape(input_shape)

    def compute_output_shape(self, input_shape):
        output_len = input_shape[2]
        if self.return_attention:
            return [(input_shape[0], output_len), (input_shape[0], input_shape[1])]
        return (input_shape[0], output_len)

    def compute_mask(self, input, input_mask=None):
        if isinstance(input_mask, list):
            return [None] * len(input_mask)
        else:
            return None

In [0]:
def model_gru_attn():
  inp = Input(shape=(max_len,))
  x = Embedding(num_words,embed_size,embeddings_initializer=Constant(embeddings),input_length=max_len,trainable=False)(inp)
  x = SpatialDropout1D(0.1)(x)
  x = Bidirectional(CuDNNLSTM(50, return_sequences=True))(x)
  x = Bidirectional(CuDNNGRU(50, return_sequences=True))(x)
  avg_pool = GlobalAveragePooling1D()(x)
  max_pool = GlobalMaxPooling1D()(x)
  last = Lambda(lambda t: t[:, -1])(x)
  attn = AttentionWeightedAverage()(x)
  conc = concatenate([avg_pool,  max_pool,last,attn])
  outp = Dense(1, activation="sigmoid")(conc)    
  model = Model(inputs=inp, outputs=outp)
  model.compile(loss=BinaryFocalLoss(gamma=2), optimizer=opt, metrics=['accuracy'])
  return model

##TCN

In [0]:
from tcn import TCN

def wave_net_activation(x):
    # type: (Layer) -> Layer
    """This method defines the activation used for WaveNet
    described in https://deepmind.com/blog/wavenet-generative-model-raw-audio/
    Args:
        x: The layer we want to apply the activation to
    Returns:
        A new layer with the wavenet activation applied
    """
    tanh_out = Activation('tanh')(x)
    sigm_out = Activation('sigmoid')(x)
    return keras.layers.multiply([tanh_out, sigm_out])

def model_tcn(embedding_matrix):
    
    inp = Input(shape=(max_len,))
    x = Embedding(num_words,embed_size,embeddings_initializer=Constant(embeddings),input_length=max_len,trainable=False)(inp)
    x = SpatialDropout1D(0.1)(x)
    x = TCN(128,dilations = [1, 2, 4], return_sequences=True ,name = 'tnc1')(x)
    x = wave_net_activation(x)
    x = TCN(64,dilations = [1, 2, 4], return_sequences=True, name = 'tnc2')(x)
    x = wave_net_activation(x)
    #x = TCN(32,dilations = [1, 2, 4], return_sequences=True, activation = 'wavenet',name = 'tnc3')(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    
    conc = concatenate([avg_pool, max_pool])
    conc = Dense(64, activation="relu")(conc)
    conc = Dense(32, activation="relu")(conc)

    conc = Dropout(0.1)(conc)
    outp = Dense(1, activation="sigmoid")(conc)    
    model = Model(inputs=inp, outputs=outp)
    model.compile(loss=BinaryFocalLoss(gamma=2), optimizer=opt, metrics=['accuracy'])
    
    return model

In [0]:
import numpy as np
class_weights = class_weight.compute_class_weight('balanced',np.unique(labels),labels)
class_weights=dict(enumerate(class_weights))

## VDCNN

In [0]:
#https://github.com/mukesh-mehta/VDCNN/blob/master/toxic.ipynb
def ConvolutionalBlock(input_shape, num_filters):
    model=Sequential()

    #1st conv layer
    model.add(Conv1D(filters = num_filters, kernel_size = 3, strides = 1, padding = "same", input_shape = input_shape))
    model.add(BatchNormalization())
    model.add(Activation("relu"))

    #2nd conv layer
    model.add(Conv1D(filters = num_filters, kernel_size = 3, strides = 1, padding = "same"))
    model.add(BatchNormalization())
    model.add(Activation("relu"))

    return model

#https://www.tensorflow.org/api_docs/python/tf/nn/top_k
def top_kmax(x):
    x=tf.transpose(x, [0, 2, 1])
    k_max = tf.nn.top_k(x, k=top_k)
    return tf.reshape(k_max[0], (-1, num_filters[-1]*top_k))

def conv_shape(conv):
    return conv.get_shape().as_list()[1:]


In [0]:
def model_vdcnn(num_filters, num_classes, sequence_max_length, num_chars, embedding_size, top_k, learning_rate=0.001):
    
    inputs=Input(shape=(sequence_max_length, ), dtype='int32', name='input')
    
    embedded_seq = Embedding(num_words,embed_size,embeddings_initializer=Constant(embeddings),input_length=max_len,trainable=False)(inputs)
    embedded_seq = BatchNormalization()(embedded_seq)
    #1st Layer
    conv = Conv1D(filters=64, kernel_size=3, strides=2, padding="same")(embedded_seq)
    
    #ConvBlocks
    for i in range(len(num_filters)):
        conv = ConvolutionalBlock(conv_shape(conv), num_filters[i])(conv)
        conv = MaxPooling1D(pool_size=3, strides=2, padding="same")(conv)
        
    def _top_k(x):
        x = tf.transpose(x, [0, 2, 1])
        k_max = tf.nn.top_k(x, k=top_k)
        return tf.reshape(k_max[0], (-1, num_filters[-1] * top_k))
    
    k_max = Lambda(_top_k, output_shape=(num_filters[-1] * top_k,))(conv)
    
    #fully connected layers
    # in original paper they didn't used dropouts
    fc1=Dense(100, activation='relu', kernel_initializer='he_normal')(k_max)
    fc1=Dropout(0.3)(fc1)
    fc2=Dense(20, activation='relu', kernel_initializer='he_normal')(fc1)
    fc2=Dropout(0.3)(fc2)
    out=Dense(1, activation='sigmoid')(fc2)
    
    
    model = Model(inputs=inputs, outputs=out)
    model.compile(optimizer=opt, loss=BinaryFocalLoss(gamma=2), metrics=['accuracy'])
    
    return model

In [0]:
num_filters = [64, 128, 256, 512]
vdcnn_model=model_vdcnn(num_filters=num_filters, num_classes=1,num_chars=69, sequence_max_length=max_len,embedding_size=16,top_k=3)

##KIM

In [0]:
conv_filters=128
def model_kim():
  inp = Input(shape=(max_len,))
  emb = Embedding(num_words,embed_size,embeddings_initializer=Constant(embeddings),input_length=max_len,trainable=False)(inp)
  # Specify each convolution layer and their kernel siz i.e. n-grams 
  conv1_1 = Conv1D(filters=conv_filters, kernel_size=3)(emb)
  btch1_1 = BatchNormalization()(conv1_1)
  drp1_1  = Dropout(0.2)(btch1_1)
  actv1_1 = Activation('relu')(drp1_1)
  glmp1_1 = GlobalMaxPooling1D()(actv1_1)

  conv1_2 = Conv1D(filters=conv_filters, kernel_size=4)(emb)
  btch1_2 = BatchNormalization()(conv1_2)
  drp1_2  = Dropout(0.2)(btch1_2)
  actv1_2 = Activation('relu')(drp1_2)
  glmp1_2 = GlobalMaxPooling1D()(actv1_2)

  conv1_3 = Conv1D(filters=conv_filters, kernel_size=5)(emb)
  btch1_3 = BatchNormalization()(conv1_3)
  drp1_3  = Dropout(0.2)(btch1_3)
  actv1_3 = Activation('relu')(drp1_3)
  glmp1_3 = GlobalMaxPooling1D()(actv1_3)

  conv1_4 = Conv1D(filters=conv_filters, kernel_size=6)(emb)
  btch1_4 = BatchNormalization()(conv1_4)
  drp1_4  = Dropout(0.2)(btch1_4)
  actv1_4 = Activation('relu')(drp1_4)
  glmp1_4 = GlobalMaxPooling1D()(actv1_4)

  # Gather all convolution layers
  cnct = concatenate([glmp1_1, glmp1_2, glmp1_3, glmp1_4], axis=1)
  drp1 = Dropout(0.2)(cnct)

  dns1  = Dense(32, activation='relu')(drp1)
  btch1 = BatchNormalization()(dns1)
  drp2  = Dropout(0.2)(btch1)

  out = Dense(1, activation='sigmoid')(drp2)   
  model = Model(inputs=inp, outputs=out)
  model.compile(loss=BinaryFocalLoss(gamma=2), optimizer=opt, metrics=['accuracy'])
  return model


## CharCNN

In [40]:

# parameter
input_size = 200
vocab_size = len(ch_tokenize.word_index)
embedding_size = len(alphabet)+1
conv_layers = [[256, 7, 3],
               [256, 7, 3],
               [256, 3, -1],
               [256, 3, -1],
               [256, 3, -1],
               [256, 3, 3]]

fully_connected_layers = [1024, 1024]
dropout_p = 0.5


# Embedding weights
embedding_weights = []  
embedding_weights.append(np.zeros(vocab_size))  

for char, i in ch_tokenize.word_index.items():  
    onehot = np.zeros(vocab_size)
    onehot[i - 1] = 1
    embedding_weights.append(onehot)

embedding_weights = np.array(embedding_weights)
print('Load')

# Embedding layer Initialization
embedding_layer = Embedding(vocab_size + 1,
                            embedding_size,
                            input_length=input_size,
                            weights=[embedding_weights])

# Model Construction
# Input
inputs = Input(shape=(input_size,), name='input', dtype='int64')  # shape=(?, 1014)
# Embedding
x = embedding_layer(inputs)
# Conv
for filter_num, filter_size, pooling_size in conv_layers:
    x = Conv1D(filter_num, filter_size)(x)
    x = Activation('relu')(x)
    if pooling_size != -1:
        x = MaxPooling1D(pool_size=pooling_size)(x)  # Final shape=(None, 34, 256)
x = Flatten()(x)  # (None, 8704)
# Fully connected layers
for dense_size in fully_connected_layers:
    x = Dense(dense_size, activation='relu')(x)  # dense_size == 1024
    x = Dropout(dropout_p)(x)
# Output Layer
predictions = Dense(1, activation='sigmoid')(x)
# Build model
char_model = Model(inputs=inputs, outputs=predictions)
char_model.compile(loss=BinaryFocalLoss(gamma=2), optimizer=opt, metrics=['accuracy'])  # Adam, categorical_crossentropy

Load


# Train model

##GRU

In [41]:
gru_model=model_gru()
gru_model.fit(x_train,y_train,batch_size=batch_size,epochs=10,verbose=2,class_weight=class_weights,validation_data=(x_val,y_val),callbacks= [clr,])

Train on 2516 samples, validate on 444 samples
Epoch 1/10
 - 10s - loss: 0.1124 - acc: 0.8645 - val_loss: 0.0992 - val_acc: 0.8694
Epoch 2/10
 - 3s - loss: 0.0938 - acc: 0.8824 - val_loss: 0.0757 - val_acc: 0.9032
Epoch 3/10
 - 3s - loss: 0.0774 - acc: 0.9046 - val_loss: 0.0746 - val_acc: 0.9077
Epoch 4/10
 - 3s - loss: 0.0714 - acc: 0.9138 - val_loss: 0.0738 - val_acc: 0.9054
Epoch 5/10
 - 3s - loss: 0.0671 - acc: 0.9169 - val_loss: 0.0758 - val_acc: 0.9032
Epoch 6/10
 - 3s - loss: 0.0582 - acc: 0.9285 - val_loss: 0.0818 - val_acc: 0.9077
Epoch 7/10
 - 3s - loss: 0.0503 - acc: 0.9368 - val_loss: 0.0878 - val_acc: 0.8941
Epoch 8/10
 - 3s - loss: 0.0430 - acc: 0.9479 - val_loss: 0.0874 - val_acc: 0.9144
Epoch 9/10
 - 3s - loss: 0.0349 - acc: 0.9587 - val_loss: 0.1437 - val_acc: 0.8986
Epoch 10/10
 - 3s - loss: 0.0317 - acc: 0.9603 - val_loss: 0.1125 - val_acc: 0.8896


## GRU attention

In [42]:
attn_gru_model=model_gru_attn()
attn_gru_model.fit(x_train,y_train,batch_size=batch_size,epochs=10,verbose=2,class_weight=class_weights,validation_data=(x_val,y_val),callbacks= [clr,])

Train on 2516 samples, validate on 444 samples
Epoch 1/10
 - 4s - loss: 0.0996 - acc: 0.8748 - val_loss: 0.0765 - val_acc: 0.8986
Epoch 2/10
 - 3s - loss: 0.0804 - acc: 0.9066 - val_loss: 0.0741 - val_acc: 0.9144
Epoch 3/10
 - 3s - loss: 0.0693 - acc: 0.9205 - val_loss: 0.0750 - val_acc: 0.9122
Epoch 4/10
 - 3s - loss: 0.0602 - acc: 0.9281 - val_loss: 0.0777 - val_acc: 0.9122
Epoch 5/10
 - 3s - loss: 0.0522 - acc: 0.9352 - val_loss: 0.0864 - val_acc: 0.9099
Epoch 6/10
 - 3s - loss: 0.0419 - acc: 0.9471 - val_loss: 0.1056 - val_acc: 0.8446
Epoch 7/10
 - 3s - loss: 0.0339 - acc: 0.9575 - val_loss: 0.1092 - val_acc: 0.9009
Epoch 8/10
 - 3s - loss: 0.0235 - acc: 0.9730 - val_loss: 0.1251 - val_acc: 0.9009
Epoch 9/10
 - 3s - loss: 0.0179 - acc: 0.9813 - val_loss: 0.1492 - val_acc: 0.8874
Epoch 10/10
 - 3s - loss: 0.0162 - acc: 0.9781 - val_loss: 0.1385 - val_acc: 0.8986


##TCN

In [43]:
tcn_model=model_tcn(embeddings)
tcn_model.fit(x_train,y_train,batch_size=batch_size,epochs=10,verbose=2,class_weight=class_weights,validation_data=(x_val,y_val),callbacks= [clr,])

Train on 2516 samples, validate on 444 samples
Epoch 1/10
 - 4s - loss: 0.1361 - acc: 0.8581 - val_loss: 0.1046 - val_acc: 0.8694
Epoch 2/10
 - 1s - loss: 0.1106 - acc: 0.8704 - val_loss: 0.1030 - val_acc: 0.8694
Epoch 3/10
 - 1s - loss: 0.1089 - acc: 0.8704 - val_loss: 0.1029 - val_acc: 0.8694
Epoch 4/10
 - 1s - loss: 0.1105 - acc: 0.8704 - val_loss: 0.1029 - val_acc: 0.8694
Epoch 5/10
 - 1s - loss: 0.1074 - acc: 0.8704 - val_loss: 0.1031 - val_acc: 0.8694
Epoch 6/10
 - 1s - loss: 0.1090 - acc: 0.8704 - val_loss: 0.1043 - val_acc: 0.8694
Epoch 7/10
 - 1s - loss: 0.1073 - acc: 0.8704 - val_loss: 0.1032 - val_acc: 0.8694
Epoch 8/10
 - 1s - loss: 0.1099 - acc: 0.8704 - val_loss: 0.1029 - val_acc: 0.8694
Epoch 9/10
 - 1s - loss: 0.1089 - acc: 0.8704 - val_loss: 0.1038 - val_acc: 0.8694
Epoch 10/10
 - 1s - loss: 0.1066 - acc: 0.8704 - val_loss: 0.1031 - val_acc: 0.8694


##VDCNN

In [44]:
vdcnn_model.fit(x_train,y_train,batch_size=batch_size,epochs=10,verbose=2,class_weight=class_weights,validation_data=(x_val,y_val),callbacks= [clr,])

Train on 2516 samples, validate on 444 samples
Epoch 1/10
 - 5s - loss: 2.3038 - acc: 0.8637 - val_loss: 2.1055 - val_acc: 0.8694
Epoch 2/10
 - 3s - loss: 2.2828 - acc: 0.8704 - val_loss: 2.1055 - val_acc: 0.8694
Epoch 3/10
 - 3s - loss: 2.2559 - acc: 0.8704 - val_loss: 2.1055 - val_acc: 0.8694
Epoch 4/10
 - 3s - loss: 2.2403 - acc: 0.8704 - val_loss: 2.1055 - val_acc: 0.8694
Epoch 5/10
 - 3s - loss: 2.2718 - acc: 0.8704 - val_loss: 2.1055 - val_acc: 0.8694
Epoch 6/10
 - 3s - loss: 2.3294 - acc: 0.8704 - val_loss: 2.1055 - val_acc: 0.8694
Epoch 7/10
 - 3s - loss: 0.6020 - acc: 0.8676 - val_loss: 0.1223 - val_acc: 0.8694
Epoch 8/10
 - 3s - loss: 0.1109 - acc: 0.8700 - val_loss: 0.1033 - val_acc: 0.8694
Epoch 9/10
 - 3s - loss: 0.1074 - acc: 0.8704 - val_loss: 0.1032 - val_acc: 0.8694
Epoch 10/10
 - 3s - loss: 0.1072 - acc: 0.8704 - val_loss: 0.1030 - val_acc: 0.8694


##KIM

In [45]:
kim_model=model_kim()
kim_model.fit(x_train,y_train,batch_size=batch_size,epochs=10,verbose=2,class_weight=class_weights,validation_data=(x_val,y_val),callbacks= [clr,])

Train on 2516 samples, validate on 444 samples
Epoch 1/10
 - 3s - loss: 0.1258 - acc: 0.8450 - val_loss: 0.1210 - val_acc: 0.8491
Epoch 2/10
 - 1s - loss: 0.1050 - acc: 0.8653 - val_loss: 0.1041 - val_acc: 0.8694
Epoch 3/10
 - 1s - loss: 0.0971 - acc: 0.8696 - val_loss: 0.1098 - val_acc: 0.8694
Epoch 4/10
 - 1s - loss: 0.0801 - acc: 0.8859 - val_loss: 0.1134 - val_acc: 0.8694
Epoch 5/10
 - 1s - loss: 0.0762 - acc: 0.8986 - val_loss: 0.1004 - val_acc: 0.8761
Epoch 6/10
 - 1s - loss: 0.0627 - acc: 0.9141 - val_loss: 0.1112 - val_acc: 0.8761
Epoch 7/10
 - 1s - loss: 0.0514 - acc: 0.9277 - val_loss: 0.0882 - val_acc: 0.8919
Epoch 8/10
 - 1s - loss: 0.0497 - acc: 0.9348 - val_loss: 0.1003 - val_acc: 0.8941
Epoch 9/10
 - 1s - loss: 0.0419 - acc: 0.9444 - val_loss: 0.1082 - val_acc: 0.8874
Epoch 10/10
 - 1s - loss: 0.0363 - acc: 0.9535 - val_loss: 0.1015 - val_acc: 0.8874


## CharCNN

In [46]:
x_train_char,x_val_char,y_train_char,y_val_char=train_test_split(train_data,labels,test_size=test_size,stratify=labels,shuffle=True,random_state=2020)
char_model.fit(x_train_char,y_train_char,batch_size=batch_size,epochs=10,verbose=2,class_weight=class_weights,validation_data=(x_val_char,y_val_char),callbacks= [clr,])

Train on 2516 samples, validate on 444 samples
Epoch 1/10
 - 2s - loss: 2.2919 - acc: 0.8688 - val_loss: 2.1055 - val_acc: 0.8694
Epoch 2/10
 - 1s - loss: 2.3498 - acc: 0.8704 - val_loss: 2.1055 - val_acc: 0.8694
Epoch 3/10
 - 1s - loss: 2.2986 - acc: 0.8704 - val_loss: 2.1055 - val_acc: 0.8694
Epoch 4/10
 - 1s - loss: 2.2757 - acc: 0.8704 - val_loss: 2.1055 - val_acc: 0.8694
Epoch 5/10
 - 1s - loss: 2.2422 - acc: 0.8704 - val_loss: 2.1055 - val_acc: 0.8694
Epoch 6/10
 - 1s - loss: 2.2815 - acc: 0.8704 - val_loss: 2.1055 - val_acc: 0.8694
Epoch 7/10
 - 1s - loss: 2.2835 - acc: 0.8704 - val_loss: 2.1055 - val_acc: 0.8694
Epoch 8/10
 - 1s - loss: 2.2977 - acc: 0.8704 - val_loss: 2.1055 - val_acc: 0.8694
Epoch 9/10
 - 1s - loss: 2.2688 - acc: 0.8704 - val_loss: 2.1055 - val_acc: 0.8694
Epoch 10/10
 - 1s - loss: 2.3092 - acc: 0.8704 - val_loss: 2.1055 - val_acc: 0.8694


# Validate Models

##GRU

In [47]:
y_pred = gru_model.predict(x_val, batch_size=batch_size, verbose=1)
y_pred = (y_pred > 0.5)
print(classification_report(y_val, y_pred))
gru_f1=f1_score(y_val, y_pred,average='macro')

444/444 [==============================] - 1s 2ms/step
              precision    recall  f1-score   support

           0       0.92      0.95      0.94       386
           1       0.60      0.47      0.52        58

    accuracy                           0.89       444
   macro avg       0.76      0.71      0.73       444
weighted avg       0.88      0.89      0.88       444



## GRU attention

In [48]:
y_pred = attn_gru_model.predict(x_val, batch_size=batch_size, verbose=1)
y_pred = (y_pred > 0.5)
print(classification_report(y_val, y_pred))
gru_atten_f1=f1_score(y_val, y_pred,average='macro')

444/444 [==============================] - 1s 2ms/step
              precision    recall  f1-score   support

           0       0.92      0.96      0.94       386
           1       0.66      0.47      0.55        58

    accuracy                           0.90       444
   macro avg       0.79      0.71      0.74       444
weighted avg       0.89      0.90      0.89       444



##TCN

In [49]:
y_pred = tcn_model.predict(x_val, batch_size=batch_size, verbose=1)
y_pred = (y_pred > 0.5)
print(classification_report(y_val, y_pred))
tcn_f1=f1_score(y_val, y_pred,average='macro')

444/444 [==============================] - 1s 2ms/step
              precision    recall  f1-score   support

           0       0.87      1.00      0.93       386
           1       0.00      0.00      0.00        58

    accuracy                           0.87       444
   macro avg       0.43      0.50      0.47       444
weighted avg       0.76      0.87      0.81       444



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## VDCNN

In [50]:
y_pred = vdcnn_model.predict(x_val, batch_size=batch_size, verbose=1)
y_pred = (y_pred > 0.5)
print(classification_report(y_val, y_pred))
vdcnn_f1=f1_score(y_val, y_pred,average='macro')

444/444 [==============================] - 1s 2ms/step
              precision    recall  f1-score   support

           0       0.87      1.00      0.93       386
           1       0.00      0.00      0.00        58

    accuracy                           0.87       444
   macro avg       0.43      0.50      0.47       444
weighted avg       0.76      0.87      0.81       444



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


##KIM

In [51]:
y_pred = kim_model.predict(x_val, batch_size=batch_size, verbose=1)
y_pred = (y_pred > 0.5)
print(classification_report(y_val, y_pred))
kim_f1=f1_score(y_val, y_pred,average='macro')

444/444 [==============================] - 1s 2ms/step
              precision    recall  f1-score   support

           0       0.90      0.97      0.94       386
           1       0.64      0.31      0.42        58

    accuracy                           0.89       444
   macro avg       0.77      0.64      0.68       444
weighted avg       0.87      0.89      0.87       444



## CharCNN

In [52]:
y_pred = char_model.predict(x_val_char, batch_size=batch_size, verbose=1)
y_pred = (y_pred > 0.5)
print(classification_report(y_val_char, y_pred))
char_f1=f1_score(y_val, y_pred,average='macro')

444/444 [==============================] - 1s 2ms/step
              precision    recall  f1-score   support

           0       0.87      1.00      0.93       386
           1       0.00      0.00      0.00        58

    accuracy                           0.87       444
   macro avg       0.43      0.50      0.47       444
weighted avg       0.76      0.87      0.81       444



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Result

In [53]:
print('f1 score of count vec' ,cv_f1)
print('f1 score of word tfidf' ,tfw_f1)
print('f1 score of char tfidf' ,tfc_f1)

print('f1 score of RNN' ,gru_f1)
print('f1 score of gated attention', gru_atten_f1)
print('f1 score of tcn',tcn_f1)
print('f1 score of vdcnn', vdcnn_f1)
print('f1 score of kim',kim_f1)
print('f1 score of char cnn',char_f1)

f1 score of count vec 0.7583593283334439
f1 score of word tfidf 0.5159276071917062
f1 score of char tfidf 0.5318284777005389
f1 score of RNN 0.7309257312472945
f1 score of gated attention 0.744210162461113
f1 score of tcn 0.4650602409638554
f1 score of vdcnn 0.4650602409638554
f1 score of kim 0.678130255755959
f1 score of char cnn 0.4650602409638554


# Ensemble

we will choose only those, having f1 greater than 0.7. Once they are selected, we will decode labels, and keep label according to mod

In [54]:
y_test_cv = cv_classifier.predict_proba(X_test_cv)[:,1]
# y_test_tfw = tfw_classifier.predict_proba(test_word_features)[:,1]
# y_test_tfc = tfc_classifier.predict_proba(test_char_features)[:,1]

y_test_gru = gru_model.predict(X_test_sequences, batch_size=batch_size, verbose=1).ravel()
y_test_gru_atten = attn_gru_model.predict(X_test_sequences, batch_size=batch_size, verbose=1).ravel()
# y_test_tcn = tcn_model.predict(X_test_sequences, batch_size=batch_size, verbose=1).ravel()
# y_test_vdcnn = vdcnn_model.predict(X_test_sequences, batch_size=batch_size, verbose=1).ravel()
y_test_kim = kim_model.predict(X_test_sequences, batch_size=batch_size, verbose=1).ravel()
# y_test_char = char_model.predict(X_test_sequences, batch_size=batch_size, verbose=1).ravel()
# # # 

329/329 [==============================] - 0s 152us/step


In [60]:
f=np.array([0.75,0.73,0.74,0.67])
f/np.sum(f)

array([0.25951557, 0.25259516, 0.25605536, 0.23183391])

In [0]:
#i am bit changing the distribution in order to give more weightage to ml than dl
y_test=0.3*y_test_cv+0.25*y_test_gru+0.25*y_test_gru_atten+0.2*y_test_kim

In [0]:
def decode(y_test):
  y_test[y_test>0.5]=1
  y_test[y_test<0.5]=0
  y_test=y_test.astype('int16').ravel()

  y_test=le.inverse_transform(y_test)
  y_test=pd.DataFrame(y_test,columns=['label'])
  y_test=pd.concat([ids, y_test['label']], axis=1)
  return y_test

In [64]:
y_test=decode(y_test)
y_test.head()



,id,label
0,1382,NOT
1,1384,NOT
2,547,NOT
3,1269,NOT
4,1695,NOT


# Submit file

In [0]:
y_test.to_csv('/content/weighted.csv',index=False,header=None)


# Reset

In [0]:
# import keras
# keras.backend.clear_session()